> ### Importando packages

In [1]:
# import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install pandera
# !{sys.executable} -m pip install numpy

In [2]:
import pandas as pd
import pandera as pa # A Statistical Data Validation Toolkit for Pandas
import numpy as np

> ### Lendo todos os dataset

In [3]:
df_clientes = pd.read_csv("./data/clean_customers_dataset.csv")
df_vendedores = pd.read_csv("./data/clean_sellers_dataset.csv")
df_itens_pedidos = pd.read_csv("./data/clean_order_items_dataset.csv")
df_pedidos = pd.read_csv('./data/clean_orders_dataset.csv')

> ### Fundindo datasets

In [4]:
df_logistica = pd.merge(df_pedidos, df_clientes, on='id_cliente', how='left')

In [5]:
df_logistica = pd.merge(df_logistica, df_itens_pedidos)

In [6]:
df_logistica = pd.merge(df_logistica, df_vendedores, on='id_vendedor')

In [7]:
assert df_pedidos.id_pedido.count() == df_logistica.id_pedido.count() # aferindo unicidade do dados iniciais

> ### Tratando as colunas de datas

In [8]:
df_logistica.dtypes

id_pedido                      object
id_cliente                     object
status_pedido                  object
data_hora_compra               object
data_hora_aprovacao            object
data_hora_envio                object
data_hora_entrega              object
data_hora_previsao_entrega     object
cidade_cliente                 object
estado_cliente                 object
quantidade_itens                int64
id_produto                     object
id_vendedor                    object
data_limite_envio              object
preco_unitario                float64
valor_frete                   float64
cidade_vendedor                object
estado_vendedor                object
dtype: object

In [9]:
# convertendo datas de string para datetime
df_logistica['data_hora_compra'] = pd.to_datetime(df_logistica['data_hora_compra'], format='%Y-%m-%d %H:%M:%S')
df_logistica['data_hora_aprovacao'] = pd.to_datetime(df_logistica['data_hora_aprovacao'], format='%Y-%m-%d %H:%M:%S')
df_logistica['data_hora_envio'] = pd.to_datetime(df_logistica['data_hora_envio'], format='%Y-%m-%d %H:%M:%S')
df_logistica['data_limite_envio'] = pd.to_datetime(df_logistica['data_hora_previsao_entrega'], format='%Y-%m-%d %H:%M:%S')
df_logistica['data_hora_entrega'] = pd.to_datetime(df_logistica['data_hora_entrega'], format='%Y-%m-%d %H:%M:%S')
df_logistica['data_hora_previsao_entrega'] = pd.to_datetime(df_logistica['data_hora_previsao_entrega'], format='%Y-%m-%d %H:%M:%S')

> ### Acrecentando colunas

In [10]:
capitais = ["rio branco", "macapa", "manaus", "belem", "porto velho", "boa vista", "palmas", "macieo", "salvador", "fortaleza", "sao luis", "joao pessoa", "recife", "teresina", "natal", "aracaju", "goiania", "cuiaba", "campo grande", "brasilia", "vitoria", "belo horizonte", "sao paulo", "rio de janeiro", "curitiba", "porto Alegre", "florianopolis"]

# mapeando condições das regiões 
regioes_cliente = {
    "norte" : df_logistica.estado_cliente.isin(["AM", "RR", "AP", "PA", "TO", "RO", "AC"]),
    "nordeste" : df_logistica.estado_cliente.isin(["MA", "PI", "CE", "RN", "PE", "PB", "SE", "AL", "BA"]),
    "centro" : df_logistica.estado_cliente.isin(["MT", "MS", "GO", "DF"]),
    "sudeste" : df_logistica.estado_cliente.isin(["SP", "RJ", "MG", "ES"]),
    "sul" : df_logistica.estado_cliente.isin(["RS", "PR", "SC"])
}
regioes_vendedor = {
    "norte" : df_logistica.estado_vendedor.isin(["AM", "RR", "AP", "PA", "TO", "RO", "AC"]),
    "nordeste" : df_logistica.estado_vendedor.isin(["MA", "PI", "CE", "RN", "PE", "PB", "SE", "AL", "BA"]),
    "centro" : df_logistica.estado_vendedor.isin(["MT", "MS", "GO", "DF"]),
    "sudeste" : df_logistica.estado_vendedor.isin(["SP", "RJ", "MG", "ES"]),
    "sul" : df_logistica.estado_vendedor.isin(["RS", "PR", "SC"])
}

In [11]:
# colunas referentes à duração cumprimento dos prazos de cada etapa no sistema de logística
# ato da compra
df_logistica['delta_compra_aprovação'] = df_logistica['data_hora_aprovacao'] - df_logistica['data_hora_compra']

# da compra ao envio
df_logistica['delta_aprovação_envio'] = (df_logistica['data_hora_envio'] - df_logistica['data_hora_aprovacao']).dt.days
df_logistica['enviado_no_prazo'] = df_logistica['data_limite_envio'] > df_logistica['data_hora_envio']
df_logistica['delta_envio_prazo'] = (df_logistica['data_limite_envio'] - df_logistica['data_hora_envio']).dt.days

# do envio à entrega
df_logistica['delta_envio_entrega'] = (df_logistica['data_hora_entrega'] - df_logistica['data_hora_envio']).dt.days
df_logistica['entregue_no_prazo'] = df_logistica['data_hora_previsao_entrega'] > df_logistica['data_hora_entrega']
df_logistica['delta_entrega_prazo'] = (df_logistica['data_hora_previsao_entrega'] - df_logistica['data_hora_entrega']).dt.days

# colunas referentes ao valores totais cobrados pelo frete
df_logistica['valor_total_frete'] = df_logistica['valor_frete'] * df_logistica['quantidade_itens']
df_logistica['valor_total_pedido'] = df_logistica['preco_unitario'] * df_logistica['quantidade_itens']
df_logistica['proporcao_pedido_frete'] = (df_logistica['valor_total_frete'] / df_logistica['valor_total_pedido'])

# colunas referentes ao fluxo no sistema de logística
# capital x capital, capital x interior, interior x capital, interior x interior
df_logistica['cliente_capital'] = df_logistica.cidade_cliente.isin(capitais)
df_logistica['vendedor_capital'] = df_logistica.cidade_vendedor.isin(capitais)
# região
df_logistica['regiao_cliente'] = np.select(regioes_cliente.values(), regioes_cliente.keys())
df_logistica['regiao_vendedor'] = np.select(regioes_vendedor.values(), regioes_vendedor.keys())

# data
df_logistica["data_hora_compra"] = df_logistica["data_hora_compra"].dt.strftime('%Y-%m')
df_logistica['data_hora_aprovacao'] = df_logistica["data_hora_aprovacao"].dt.strftime('%Y-%m')
df_logistica['data_hora_envio'] = df_logistica["data_hora_envio"].dt.strftime('%Y-%m')
df_logistica['data_limite_envio'] = df_logistica["data_limite_envio"].dt.strftime('%Y-%m')
df_logistica['data_hora_entrega'] = df_logistica["data_hora_entrega"].dt.strftime('%Y-%m')
df_logistica['data_hora_previsao_entrega'] = df_logistica["data_hora_previsao_entrega"].dt.strftime('%Y-%m')

> ### Dropando colunas e reordenando

In [12]:
df_logistica.drop(['id_cliente', 'id_vendedor', 'id_produto', 'status_pedido', 'quantidade_itens', 'preco_unitario', 'valor_frete'], axis=1, inplace = True)

In [13]:
df_logistica = df_logistica[["id_pedido", "cidade_cliente", "estado_cliente", "cliente_capital", "regiao_cliente", "cidade_vendedor", "estado_vendedor", "vendedor_capital", "regiao_vendedor", "data_hora_compra", "data_hora_aprovacao", "delta_compra_aprovação", "data_hora_envio", "delta_aprovação_envio", "data_limite_envio", "delta_envio_prazo", "enviado_no_prazo", "data_hora_entrega", "delta_envio_entrega", "data_hora_previsao_entrega", "delta_entrega_prazo", "entregue_no_prazo", "valor_total_frete", "valor_total_pedido", "proporcao_pedido_frete"]]

In [14]:
df_logistica.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96455 entries, 0 to 96454
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   id_pedido                   96455 non-null  object         
 1   cidade_cliente              96455 non-null  object         
 2   estado_cliente              96455 non-null  object         
 3   cliente_capital             96455 non-null  bool           
 4   regiao_cliente              96455 non-null  object         
 5   cidade_vendedor             96455 non-null  object         
 6   estado_vendedor             96455 non-null  object         
 7   vendedor_capital            96455 non-null  bool           
 8   regiao_vendedor             96455 non-null  object         
 9   data_hora_compra            96455 non-null  object         
 10  data_hora_aprovacao         96455 non-null  object         
 11  delta_compra_aprovação      96455 non-nul

> ### Exportando para clean_sellers_dataset.csv

In [15]:
df_logistica.head()

,id_pedido,cidade_cliente,estado_cliente,cliente_capital,regiao_cliente,cidade_vendedor,estado_vendedor,vendedor_capital,regiao_vendedor,data_hora_compra,...,delta_envio_prazo,enviado_no_prazo,data_hora_entrega,delta_envio_entrega,data_hora_previsao_entrega,delta_entrega_prazo,entregue_no_prazo,valor_total_frete,valor_total_pedido,proporcao_pedido_frete
0,e481f51cbdc54678b7cc49136f2d6af7,sao paulo,SP,True,sudeste,maua,SP,False,sudeste,2017-10,...,13,True,2017-10,6,2017-10,7,True,8.72,29.99,0.290764
1,8736140c61ea584cb4250074756d8f3b,itaquaquecetuba,SP,False,sudeste,maua,SP,False,sudeste,2017-08,...,11,True,2017-08,5,2017-08,6,True,7.79,75.90,0.102635
2,a0151737f2f0c6c0a5fd69d45f66ceea,jaguariuna,SP,False,sudeste,maua,SP,False,sudeste,2017-07,...,11,True,2017-07,5,2017-07,6,True,7.78,44.99,0.172927
3,a3bf941183211246f0d42ad757cba127,osasco,SP,False,sudeste,maua,SP,False,sudeste,2017-07,...,8,True,2017-07,2,2017-08,6,True,7.78,44.99,0.172927
4,1462290799412b71be32dd880eaf4e1b,gravatai,RS,False,sul,maua,SP,False,sudeste,2017-08,...,22,True,2017-08,6,2017-09,16,True,14.10,29.90,0.471572


In [16]:
df_logistica.to_csv("./data/logistica_dataset.csv", index = False, header=True)